In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml import command, Input
from azure.ai.ml.constants import AssetTypes


try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

ml_client = MLClient.from_config(credential=credential)

job = command(
    code="./",
    command="python train.py --training_data ${{inputs.diabetes_data}} --reg_rate ${{inputs.reg_rate}}",
    inputs={
        "diabetes_data": Input(
            type=AssetTypes.URI_FILE, 
            path="azureml:training-data-uri-file:1"
        ),
        "reg_rate": 0.01,
    },
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="ngothilinhau129152",
    display_name="diabetes-train-job",
    experiment_name="diabetes-train-experiment", 
    tags={"model_type": "LogisticRegression"}
)

returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

In [ ]:
# command job - completed, you can configure and run a sweep job.

from azure.ai.ml.sweep import Choice

command_job_for_sweep = job(
    reg_rate=Choice(values=[0.01, 0.1, 1]),
)

sweep_job = command_job_for_sweep.sweep(
    compute="ngothilinhau129152",
    sampling_algorithm="grid",
    primary_metric="training_accuracy_score",
    goal="Maximize",
)
sweep_job.experiment_name="diabetes-train-experimen-sweep"
sweep_job.set_limits(max_total_trials=4, max_concurrent_trials=2, timeout=7200)
     
returned_sweep_job = ml_client.create_or_update(sweep_job)
aml_url = returned_sweep_job.studio_url
print("Monitor your job at", aml_url)